In [ ]:
from wasmtime import Linker, Module, Store, WasiConfig

from bindings import Abe, Attribute, Ok, Policy, PolicyAxis


store = Store()
wasi_config = WasiConfig()
wasi_config.inherit_stderr()
wasi_config.inherit_stdin()
wasi_config.inherit_stdout()
store.set_wasi(wasi_config)

linker = Linker(store.engine)
linker.define_wasi()

my_module = Module.from_file(store.engine, "abe_gpsw.wasm")
abe = Abe(store, linker, my_module)


def unwrap(wrapped_result):
    """
    If the result is an Ok, return the value, otherwise raise an exception

    :param wrapped_result: The result of a function that returns a wrapped
    object
    :return: A list of strings.
    """
    if isinstance(wrapped_result, Ok):
        return wrapped_result.value
    raise Exception(wrapped_result.value)

# Encryption using an Authorization Policy

This notebook demonstrates how data can be encrypted with policy attributes. An user will only be able to decrypt data when it holds a key with the proper attributes.

This notebook also demonstrates revocation of an attribute value and how to implement forward secrecy.



## Policy

In this demo, we will create an Policy which combines 2 axes, a 'security level' and a 'department'.

An user will be able to decrypt data only if it possesses a key with a sufficient security level and the attribute for the department.


## Policy Axes

The Policy Group is defined by two Policy Axes, thus defining a 2 dimensional matrix of authorizations. An user must possess keys with attributes from these two axes to be able to decrypt files.

### Security Level Axis
The first Policy Axis is the 'Security Level' axis and is a hierarchical axis made of 5 levels: `level_1`, `level_2`, ..., `level_5`. It is hierarchical: an user being granted access to level `n` is automatically granted access to all levels below `n`.

### Department Security Axis

The second Policy Axis is the `Department` axis and is made of 4 values: `RnD`, `HR`, `MKG`, `FIN`. This axis i *not* hierarchical: granting access to an attribute of this axis to a user does not give access to any other attribute. Each attribute must be granted individually.


In [ ]:
entities_axis = PolicyAxis(
    name="Departments", 
    attributes=["RnD", "HR", "MKG", "FIN"],
    hierarchical=False
)
security_axis = PolicyAxis(
    name="Security_Level",
    attributes=["level_1", "level_2", "level_3", "level_4", "level_5"],
    hierarchical=True,
)
policy_definition = Policy(
    primary_axis=entities_axis, secondary_axis=security_axis
)


## Master Authority

The Master Authority possesses the keys for the given Policy Group:

 - a Secret Key which is used to delegate authority to "delegate authorities", which are the ones generating user keys
 - and a Public key which is used to encrypt files with proper level of security.

The second parameter fixes the maximum number of revocations of attributes (see below) for this Policy.
This number influences the number of public keys which will be ultimately generated for this Security Group and must be kept to a "reasonable" level to reduce security risks associated with multiplying the number of keys.

In [ ]:
master_key = abe.generate_master_key(store, 100, policy_definition)
master_key = unwrap(master_key)

## Delegate Authorities

The Master Authority will delegate part or all of its authority to "Delegate Authorities" (a.k.a Delegates) which are the ones generating user keys.

In this particular example, the Master Authority wil delegate its authority to 2 Delegates:

 - a Super Delegate which can issue User Keys for all Security Levels and all Departments
 - a "Level 4 Marketing and Finance Delegate" which can only generate User Keys for marketing (MKG) and finance (FIN) data of Security Level 4 and below.

In [ ]:
super_delegate = abe.generate_user_decryption_key(
    store,
    master_private_key=master_key.private_key,
    access_policy=None,  # without further specification, all attributes are delegated
    policy=master_key.policy_serialized,
)
super_delegate = unwrap(super_delegate)

level_4_mkg_fin_delegate = abe.generate_user_decryption_key(
    store,
    master_private_key=master_key.private_key,
    access_policy="(Departments::FIN || Departments::MKG) && Security_Level::level_4",
    policy=master_key.policy_serialized,
)
level_4_mkg_fin_delegate = unwrap(level_4_mkg_fin_delegate)


## User Keys

Delegate Authorities can now generate User Keys up to the level allowed by their policy. 

A marketing user with level 3 security can have its key generated by any of the Delegates.

In [ ]:
level_3_mkg_user = abe.delegate_user_decryption_key(
    store,
    delegation_key=master_key.delegation_key,
    user_decryption_key=super_delegate,
    policy=master_key.policy_serialized,
    access_policy="Departments::MKG && Security_Level::level_3",
)
level_3_mkg_user = unwrap(level_3_mkg_user)

level_3_mkg_user = abe.delegate_user_decryption_key(
    store,
    delegation_key=master_key.delegation_key,
    user_decryption_key=level_4_mkg_fin_delegate,
    policy=master_key.policy_serialized,
    access_policy="Departments::MKG && Security_Level::level_3",
)
level_3_mkg_user = unwrap(level_3_mkg_user)


However, a Delegate cannot generate user keys for which it does not have the authority


In [ ]:
try:
    level_5_user = abe.delegate_user_decryption_key(
        store,
        delegation_key=master_key.delegation_key,
        user_decryption_key=level_4_mkg_fin_delegate,
        policy=master_key.policy_serialized,
        access_policy="Departments::MKG && Security_Level::level_5",
    )
    level_5_user = unwrap(level_5_user)
except Exception as err:
    print(
        "FAILURE: as expected the level 4 marketing authority cannot generate user keys for Security Level 5"
    )


In [ ]:
try:
    HR_user = abe.delegate_user_decryption_key(
        store,
        delegation_key=master_key.delegation_key,
        user_decryption_key=level_4_mkg_fin_delegate,
        policy=master_key.policy_serialized,
        access_policy="Departments::HR && Security_Level::level_3"
    )
    HR_user = unwrap(HR_user)
except:
    print(
        "FAILURE: as expected the level 4 marketing authority cannot generate user keys for Department HR"
    )


Let us create a super user as well, which can decrypt everything

In [ ]:
super_user = abe.delegate_user_decryption_key(
    store,
    delegation_key=master_key.delegation_key,
    user_decryption_key=super_delegate,
    policy=master_key.policy_serialized,
    access_policy=None,  # no specification of attributes => all attributes
)
super_user = unwrap(super_user)


## Encryption and Decryption

Data is encrypted using the Master Authority Public Key with two attributes: one for the Security Level and one for the Department. Anyone - who has access to the Public Key - can encrypt data with any attribute combination. However, only users posessing user keys with the right combination of attributes can decrypt data.

### Real World File System: Hybrid Encryption

In a real world encrypted File System, to maximize speed and minimize space, a hybrid encryption scheme is used:
 - the content of the file is encrypted using an AES key (a different key per file)
 - the AES key is encrypted using the public key scheme


### A level 2 marketing message
Let us create an encrypted marketing message with a security level 2. 

In [ ]:
mkg_level_2_msg = abe.encrypt(
    store,
    plaintext="Marketing message level 2",
    master_public_key=master_key.public_key,
    attributes=[
        Attribute("Departments", "MKG"),
        Attribute("Security_Level", "level_2"),
    ],
    policy=master_key.policy_serialized,
    uid=bytes([1, 2, 3, 4, 5, 6, 7, 8])
)
mkg_level_2_msg = unwrap(mkg_level_2_msg)


Both are users are able to decrypt the message

In [ ]:
result = abe.decrypt(store, user_decryption_key=level_3_mkg_user, encrypted_data=mkg_level_2_msg).value
print("Marketing level 3 User succeeds: '%s'" % result)
result = abe.decrypt(store, user_decryption_key=super_user, encrypted_data=mkg_level_2_msg).value
print("Super User succeeds            : '%s'" % result)


### A level 5 marketing message
However in the case of a level 5 marketing message, only the super user will succeed:

In [ ]:
mkg_level_5_msg = abe.encrypt(
    store,
    plaintext="Marketing message level 5",
    master_public_key=master_key.public_key,
    attributes=[
        Attribute("Departments", "MKG"),
        Attribute("Security_Level", "level_5"),
    ],
    policy=master_key.policy_serialized,
    uid=bytes([1, 2, 3, 4, 5, 6, 7, 8])
)
mkg_level_5_msg = unwrap(mkg_level_5_msg)

try:
    result = abe.decrypt(store, user_decryption_key=level_3_mkg_user, encrypted_data=mkg_level_5_msg)
    result = unwrap(result)
except:
    print("As expected, Marketing level 3 User FAILS decrypting")

result = abe.decrypt(store, user_decryption_key=super_user, encrypted_data=mkg_level_5_msg).value
print("Super User succeeds         : '%s'" % result)


### A level 2 HR message
Likewise, in the case of a level 2 HR message, only the super user will succeed:

In [ ]:
hr_level_2_msg = abe.encrypt(
    store,
    plaintext="HR message level 2",
    master_public_key=master_key.public_key,
    attributes=[
        Attribute("Departments", "HR"),
        Attribute("Security_Level", "level_2"),
    ],
    policy=master_key.policy_serialized,
    uid=bytes([1, 2, 3, 4, 5, 6, 7, 8])
)
hr_level_2_msg = unwrap(hr_level_2_msg)

try:
    result = abe.decrypt(store, user_decryption_key=level_3_mkg_user, encrypted_data=hr_level_2_msg)
    result = unwrap(result)
except:
    print("As expected, Marketing level 3 User FAILS decrypting")

result = abe.decrypt(store, user_decryption_key=super_user, encrypted_data=hr_level_2_msg).value
print("Super User succeeds         : '%s'" % result)


# Revocation

At anytime the Master Authority can revoke an attribute. When that happens future encryption of data for a given attribute cannot be decrypted with keys which are not "refreshed" for that attribute.

Let us revoke the Security Level 2

In [ ]:
updated_policy = abe.rotate_attributes(
    store,
    policy=master_key.policy_serialized,
    attributes=[
        Attribute("Security_Level", "level_2"),
    ],
)
print(updated_policy)
updated_policy = unwrap(updated_policy)


We now encrypt a new marketing message at level 2


In [ ]:
new_mkg_level_2_msg = abe.encrypt(
    store,
    plaintext="New marketing message level 2",
    master_public_key=master_key.public_key,
    attributes=[
        Attribute("Departments", "MKG"),
        Attribute("Security_Level", "level_2"),
    ],
    policy=updated_policy,
    uid=bytes([1, 2, 3, 4, 5, 6, 7, 8])
)
new_mkg_level_2_msg = unwrap(new_mkg_level_2_msg)


No user, except the super user, can decrypt the message until its key is refreshed

In [ ]:
try:
    result = abe.decrypt(store, user_decryption_key=level_3_mkg_user, encrypted_data=new_mkg_level_2_msg)
    result = unwrap(result)
except:
    print("As expected, Marketing level 3 User FAILS decrypting")

result = abe.decrypt(store, user_decryption_key=super_user, encrypted_data=new_mkg_level_2_msg).value  ## try with fin_delegate
print("Super User succeeds         : '%s'" % result)


All keys need to be refresh: Delegates and Users

In [ ]:
# Delegates
level_4_mkg_fin_delegate = abe.generate_user_decryption_key(
    store,
    master_private_key=master_key.private_key,
    access_policy="(Departments::FIN || Departments::MKG) && Security_Level::level_4",
    policy=updated_policy,
)
level_4_mkg_fin_delegate = unwrap(level_4_mkg_fin_delegate)

# Users
level_3_mkg_user = abe.delegate_user_decryption_key(
    store,
    delegation_key=master_key.delegation_key,
    user_decryption_key=level_4_mkg_fin_delegate,
    policy=updated_policy,
    access_policy="Departments::MKG && Security_Level::level_3"
)
level_3_mkg_user = unwrap(level_3_mkg_user)


New messages can now be decrypted

In [ ]:
result = abe.decrypt(store, level_3_mkg_user, new_mkg_level_2_msg).value
print("Marketing level 3 User SUCCEEDS decrypting: '%s'" % result)

result = abe.decrypt(store, super_user, new_mkg_level_2_msg).value
print("Super User succeeds         : '%s'" % result)


Older messages can still be decrypted as well

In [ ]:
result = abe.decrypt(store, user_decryption_key=level_3_mkg_user, encrypted_data=mkg_level_2_msg).value
print("Marketing level 3 User SUCCEEDS decrypting: '%s'" % result)

result = abe.decrypt(store, user_decryption_key=super_user, encrypted_data=mkg_level_2_msg).value
print("Super User succeeds         : '%s'" % result)
